In [1]:
!pip3 install bitsandbytes==0.41.3
!pip3 install peft==0.12.0
!pip3 install trl==0.8.6
!pip3 install datasets==2.19.2
!pip3 install transformers==4.43.3
!pip3 install tensorboard==2.17.0
!pip3 install accelerate==0.33.0
!pip3 install sentencepiece==0.2.0

In [2]:
import os
import torch
import transformers

from datasets import load_dataset
from transformers import (
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    logging,
    BitsAndBytesConfig
)
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [3]:
dataset_name = "Vezora/Tested-22k-Python-Alpaca"
model_id = "TinyLlama/TinyLlama_v1.1"

# load tokenizer


In [4]:
def load_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(
        model_id, 
        trust_remote_code=True, 
        use_fast=False,
        add_eos_token=True,
        add_bos_token=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    return tokenizer

# Quantization config for QLoRA

In [5]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16)

# Load model

In [6]:
def load_model(model_id, is_local=False):
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        #quantization_config=bnb_config,
        local_files_only=is_local,
        device_map={"": 0})

    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model

# Test Inference Prompts

In [7]:
prompts = [
    """Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in the field of material science and engineering?""",
    """Write a Python function that returns the maximum value of the given set of integers: 1, 5, 23, and 9. The function should only use one line of code and must utilize the lambda function. Additionally, the function should be able to handle any set of integers provided as input, and should return an error message if any non-integer values are detected. The output should be displayed in a formatted table with the following columns: "Input Set", "Maximum Value". The table should also include a row for the given set of integers as well as a row for a randomly generated set of 10 integers between 1 and 100. Lastly, the function should be written in SQL and should be able to query a database table containing the input sets and their corresponding maximum values.""",
    """Create a function that takes two parameters, a number and a string, and returns an array with those two values in it. The string parameter should be checked if it is a valid string and if it contains only alphabetical characters. If the string is not valid (length less than 10 characters) or contains non-alphabetical characters, the function should return an empty array. Additionally, the number parameter should be restricted to a range between -1000 and 1000 (inclusive). The function should also check if the number is a prime number and return an empty array if it is not.""",
    """Given a string, convert it to upper case using only basic string manipulation operations. The string may contain punctuation marks, special characters, and spaces. String: This string should be in upper case!""",
    """Find the sum of the first 1000 prime numbers that are greater than 100.""",
    """Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion.""",
    """Create a function that removes duplicates from an array and returns an array of the unique values. The function should only use O(1) additional space and have a time complexity of O(n), where n is the length of the input array. The input array can contain integers, floating-point numbers, strings, and nested arrays. The output array should be sorted in descending order. Additionally, the function should handle nested arrays correctly by recursively flattening them before removing duplicates.""",
    """Create a 3x4 NumPy array of random integers from 0 to 5, where each row should have at least one unique value.""",
    """Find the index of the element 'c' in the following list, but the list may contain duplicates and the element 'c' may appear multiple times.""",
    """Write a function for finding the minimum value in a given array, with a time complexity requirement of O(n log n), where n is the length of the array."""
]

# Generate Inference

In [8]:
def generate_inference(model, tokenizer):
    pipe = pipeline(task='text-generation', model=model, tokenizer=tokenizer, max_length=256)

    for prompt in prompts:
        result = pipe(prompt)
        print("******************************* PROMPT *******************************")
        print(prompt)
        print("******************************* Inference *******************************")
        print(result[0]['generated_text'])
        print("="*100)

# Pre Fine-tuning inference

In [9]:
TOKENIZER = load_tokenizer(model_id)
model = load_model(model_id)
generate_inference(model, TOKENIZER)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


******************************* PROMPT *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in the field of material science and engineering?
******************************* Inference *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in 

******************************* PROMPT *******************************
Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion.
******************************* Inference *******************************
Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion. The 2018-19 season is now over.
The 2018-19 season is now over. The 2019-20 season will begin on September 1, 2019.
The 2018-19 season is now over. The 2019-20 season will begin on September 1, 2019. The 2019-20 season will be a 10-game season.
The 2018-19 season is now over. The 2019-20 season will begin on September 1, 2019. The 2019-20 season will be a 10-game season. The 2019-20 season will be a 10-game season.
The 2018-19 season is now over. The 2019-20 season will begin on September 1, 
***********************

# Load dataset 

In [10]:
import datasets
from datasets import Dataset

dataset = load_dataset(dataset_name)

In [11]:
def preprocess_function(example):
    """
    Formatting function returning a list of samples (kind of necessary for SFT API).
    """
    text = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    return text

# Train test split 

In [12]:
def train_test_split(dataset):
    train_val = dataset["train"].train_test_split(test_size=4000, shuffle=True, seed=42)
    train_data = train_val["train"]
    val_data = train_val["test"]

    return train_data, val_data

# Tokenize the prompts

In [13]:
def generate_and_tokenize_prompt(prompt):
    return TOKENIZER(preprocess_function(prompt))

# tokenize the train and test datasets

In [14]:
train_data, val_data = train_test_split(dataset)

tokenized_train_dataset = train_data.map(generate_and_tokenize_prompt)
tokenized_val_dataset = val_data.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/18608 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

# LoRA parameters

In [15]:
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
    "lm_head",
]

BATCH_SIZE = 64
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300

# LoRA config

In [16]:
peft_params = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

In [17]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_params)

model.print_trainable_parameters()

trainable params: 13,160,448 || all params: 1,113,208,832 || trainable%: 1.1822


# Training params

In [18]:
OUTPUT_DIR="/home/ec2-user/SageMaker/tinyllama/LoRA"
#OUTPUT_DIR="/kaggle/working/tinyllama"
training_args = TrainingArguments(
  output_dir=OUTPUT_DIR,
  num_train_epochs=3,
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  gradient_accumulation_steps=4,
  optim="paged_adamw_32bit",
  save_steps=500,
  logging_steps=500,
  learning_rate=0.0001,
  eval_strategy="steps",
  weight_decay=0.001,
  fp16=True,
  bf16=False,
  max_grad_norm=0.3,
  max_steps=-1,
  warmup_ratio=0.03,
  group_by_length=True,
  lr_scheduler_type="constant",
  report_to="tensorboard"
)

# Model trainer

In [19]:
context_length = 1024
    
trainer = SFTTrainer(
    model=model,
    #train_dataset=tokenized_train_dataset,
    #eval_dataset=tokenized_val_dataset,
    train_dataset=train_data,
    eval_dataset=val_data,
    max_seq_length=context_length,
    tokenizer=TOKENIZER,
    args=training_args,
    formatting_func=preprocess_function,
    packing=True,
    data_collator=transformers.DataCollatorForLanguageModeling(TOKENIZER, mlm=False),
)

print("Model output path: ", f"{OUTPUT_DIR}/model/fine_tuned")

history = trainer.train()
model.save_pretrained(f"{OUTPUT_DIR}/model/fine_tuned")
#TOKENIZER.save_pretrained(f"{OUTPUT_DIR}/fine_tuned")


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Model output path:  /home/ec2-user/SageMaker/tinyllama/LoRA/model/fine_tuned


Step,Training Loss,Validation Loss
500,1.309400,1.529727
1000,1.749000,1.980769
1500,7.007200,18.480494
2000,28.430900,36.679081


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pef

In [ ]:
#model_ft = load_model("/home/ec2-user/SageMaker/gpt-2/fine_tuned")
model_ft = load_model(f"{OUTPUT_DIR}/model/fine_tuned", True)
#tokenizer_ft = load_tokenizer(f"{OUTPUT_DIR}/fine_tuned")

generate_inference(model_ft,TOKENIZER)

******************************* PROMPT *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in the field of material science and engineering?
******************************* Inference *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in 

******************************* PROMPT *******************************
Create a 3x4 NumPy array of random integers from 0 to 5, where each row should have at least one unique value.
******************************* Inference *******************************
Create a 3x4 NumPy array of random integers from 0 to 5, where each row should have at least one unique value. __```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````

KeyboardInterrupt: 